# Install lib

In [1]:
!pip install ortools
!pip install deap

     |████████████████████████████████| 14.4MB 321kB/s 
     |████████████████████████████████| 1.0MB 33.6MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


     |████████████████████████████████| 163kB 11.3MB/s 


# Import lib

In [2]:
import time
import random
import numpy as np
import pandas as pd

from deap import base, tools, creator, algorithms

from ortools.algorithms import pywrapknapsack_solver

# Clone and read test case

In [3]:
%cd /content
!git clone https://github.com/likr/kplib.git

/content
Cloning into 'kplib'...
remote: Enumerating objects: 20334, done.
remote: Total 20334 (delta 0), reused 0 (delta 0), pack-reused 20334
Receiving objects: 100% (20334/20334), 142.24 MiB | 23.80 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Checking out files: 100% (20801/20801), done.


In [4]:
def ReadData(path):
    values = list()
    weights = list()

    file = open(path, 'r')
    file.readline()
    n = int(file.readline())
    capacities = int(file.readline())
    file.readline()

    for _ in range(n):
        Vi, Wi = file.readline().split()
        values.append(int(Vi))
        weights.append(int(Wi))
    #print(values)
    return values, weights, capacities

In [5]:
files = ['00Uncorrelated', '01WeaklyCorrelated', '02StronglyCorrelated', '03InverseStronglyCorrelated', 
         '04AlmostStronglyCorrelated', '05SubsetSum', '06UncorrelatedWithSimilarWeights', '07SpannerUncorrelated', 
         '08SpannerWeaklyCorrelated', '09SpannerStronglyCorrelated', '10MultipleStronglyCorrelated', '11ProfitCeiling', '12Circle']
cases = ['n00050/R01000/s000.kp', 'n00200/R01000/s049.kp', 'n01000/R01000/s099.kp', 'n05000/R10000/s049.kp', 'n10000/R10000/s099.kp']

TestCases = []

for idx in range(len(files)):
    for jdx in range(len(cases)):
        path = '/content/kplib/{}/{}'.format(files[idx], cases[jdx])
        values, weights, capacities = ReadData(path)
        TestCases.append([values, weights, capacities])

# OR-Tools

In [ ]:
def KpORTools(values, weights, capacities):
    solver = pywrapknapsack_solver.KnapsackSolver(
        pywrapknapsack_solver.KnapsackSolver.
        KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample')

        #KNAPSACK_MULTIDIMENSION_SCIP_MIP_SOLVER
        #KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER

    solver.Init(values, [weights], [capacities])
    solver.set_time_limit(60)

    Start = time.time()
    TotalValue = solver.Solve()
    RunTime = time.time() - Start
    packed_items = [RunTime] + [x for x in range(len(values)) if solver.BestSolutionContains(x)]
    #print(packed_items)
    return packed_items

In [ ]:
for i in range(len(TestCases)):
    packed_items = [KpORTools(TestCases[i][0], TestCases[i][1], TestCases[i][2])]
    pd.DataFrame(packed_items).to_csv('/content/drive/MyDrive/AI/Knapsack/KpORToolsSCIP.csv', mode='a', header= None, index= False)
    #pd.DataFrame(packed_items).to_csv('/content/drive/MyDrive/AI/Knapsack/KpORToolsBAB.csv', mode='a', header= None, index= False)

# Genetic Algorithms using DEAP

In [6]:
def KpGA(values, weights, capacities):
    def evalKnapsack(individual):
        weight = 0
        value = 0
        for i in range(len(individual)):
            if individual[i]:
                weight += weights[i]
                value += values[i]
        if weight > capacities:
            return 0, 100000000000             
        return value, weight

    random.seed(496)

    creator.create('FitnessMulti', base.Fitness, weights=(1, -1))
    creator.create('Individual', list, fitness=creator.FitnessMulti)

    toolbox = base.Toolbox()
    toolbox.register('random_bit', lambda: random.choice([False, True]))
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.random_bit, len(values))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("evaluate", evalKnapsack)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.5)
    toolbox.register("select", tools.selTournament, tournsize=3)

    MU = 25
    LAMBDA = 50
    N_GENERATIONS = 80
    P_CROSSOVER = 0.7
    P_MUTATION = 0.2
    
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    
    algorithms.eaMuPlusLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                              ngen=N_GENERATIONS, halloffame=hof, verbose=True)

    return hof[0]

In [ ]:
for i in range(len(TestCases)):
    hof = KpGA(TestCases[i][0], TestCases[i][1], TestCases[i][2])
    item = []
    for i in range(len(hof)):
        if hof[i]:
            item.append(i)
    item = [item]
    pd.DataFrame(item).to_csv('/content/drive/MyDrive/AI/Knapsack/KpGA.csv', mode='a', header= None, index= False)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals
0  	25    
1  	44    
2  	44    
3  	45    
4  	45    
5  	42    
6  	44    
7  	45    
8  	45    
9  	48    
10 	46    
11 	44    
12 	47    
13 	48    
14 	45    
15 	41    
16 	44    
17 	45    
18 	47    
19 	44    
20 	50    
21 	48    
22 	43    
23 	46    
24 	47    
25 	41    
26 	44    
27 	46    
28 	46    
29 	47    
30 	46    
31 	48    
32 	47    
33 	47    
34 	45    
35 	44    
36 	46    
37 	43    
38 	47    
39 	44    
40 	43    
41 	42    
42 	44    
43 	45    
44 	46    
45 	47    
46 	46    
47 	47    
48 	46    
49 	42    
50 	43    
51 	45    
52 	44    
53 	44    
54 	44    
55 	41    
56 	44    
57 	46    
58 	45    
59 	44    
60 	48    
61 	45    
62 	46    
63 	42    
64 	46    
65 	49    
66 	44    
67 	46    
68 	46    
69 	45    
70 	49    
71 	44    
72 	44    
73 	46    
74 	46    
75 	45    
76 	43    
77 	42    
78 	42    
79 	44    
80 	44    
gen	nevals
0  	25    
1  	45    
2  	45    
3  	49    
4  	48    
5  	48    
6  	45    
7  	45    